In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import torch
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ffedo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
df = pd.read_csv('train.csv')
df.drop(columns=['index'], inplace=True)
df

,review,sentiment
0,"Есть много причин, по которым 'Война и мир' Ль...",1
1,"Напишите 5 предложений. 1. ""Война и мир"" — это...",2
2,[Практикуйте «Другие люди»] Отзыв о «Войне и ...,1
3,Стремитесь к точному и объективному представле...,2
4,"В книге ""Война и мир"" Льва Толстого разворачив...",2
...,...,...
23995,"Вот и все, что нужно сделать: Во-первых, объяс...",1
23996,"Обязательно укажите, что именно в тексте/сюжет...",0
23997,"Кто, из героев вас заинтересовал и почему? Как...",1
23998,"Жду историй о том, как книга изменила ваше вос...",1


In [5]:
sw = stopwords.words('russian')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r'\W+', ' ', text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

In [6]:
df['review'] = df['review'].apply(lambda x: clean_text(x))
df

,review,sentiment
0,причин которым война мир льва толстого считает...,1
1,напишите 5 предложений 1 война мир это роман л...,2
2,практикуйте другие люди отзыв войне мире льва ...,1
3,стремитесь точному объективному представлению ...,2
4,книге война мир льва толстого разворачивается ...,2
...,...,...
23995,нужно сделать первых объясните заключается кни...,1
23996,обязательно укажите именно тексте сюжете персо...,0
23997,героев заинтересовал почему какова судьба толс...,1
23998,жду историй книга изменила ваше восприятие мир...,1


In [7]:
from transformers import BertTokenizer
tokenizer_path = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
max_len = 0
for sent in tqdm(df.review):

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, truncation=True, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

  0%|          | 0/24000 [00:00<?, ?it/s]

Max sentence length:  512


In [9]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):

  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        truncation=True,
        padding='max_length',
        # pad_to_max_length = True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [10]:
# input_ids = []
# attention_masks = []
# labels = df.sentiment.values

# # For every tweet...
# for tweet in tqdm(df.review):
#     # `encode_plus` will:
#     #   (1) Tokenize the sentence.
#     #   (2) Prepend the `[CLS]` token to the start.
#     #   (3) Append the `[SEP]` token to the end.
#     #   (4) Map tokens to their IDs.
#     #   (5) Pad or truncate the sentence to `max_length`
#     #   (6) Create attention masks for [PAD] tokens.
#     encoded_dict = tokenizer.encode_plus(
#                         tweet,                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                         max_length = max_len,   
#                         padding=True,
#                         truncation=True,
#                         # pad_to_max_length = True,
#                         return_attention_mask = True,   # Construct attn. masks.
#                         return_tensors = 'pt',     # Return pytorch tensors.
#                    )
    
#     # Add the encoded sentence to the list.    
#     input_ids.append(encoded_dict['input_ids'])

#     # And its attention mask (simply differentiates padding from non-padding).
#     attention_masks.append(encoded_dict['attention_mask'])

# # Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)

In [11]:
from transformers import BertForSequenceClassification
model_path = 'cointegrated/rubert-tiny'
model = BertForSequenceClassification.from_pretrained(model_path)

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
out_features = model.bert.encoder.layer[1].output.dense.out_features
out_features

312

In [13]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, e

In [14]:
model.classifier = torch.nn.Linear(312, 3)
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, e

In [15]:
X = df.review
X

0        причин которым война мир льва толстого считает...
1        напишите 5 предложений 1 война мир это роман л...
2        практикуйте другие люди отзыв войне мире льва ...
3        стремитесь точному объективному представлению ...
4        книге война мир льва толстого разворачивается ...
                               ...                        
23995    нужно сделать первых объясните заключается кни...
23996    обязательно укажите именно тексте сюжете персо...
23997    героев заинтересовал почему какова судьба толс...
23998    жду историй книга изменила ваше восприятие мир...
23999    несколько причин почему война мир льва толстог...
Name: review, Length: 24000, dtype: object

In [16]:
y = df.sentiment
y

0        1
1        2
2        1
3        2
4        2
        ..
23995    1
23996    0
23997    1
23998    1
23999    1
Name: sentiment, Length: 24000, dtype: int64

In [17]:
# from torch.utils.data import TensorDataset, random_split
# dataset = TensorDataset(input_ids, attention_masks, labels)

# # Create a 90-10 train-validation split.

# # Calculate the number of samples to include in each set.
# train_size = int(0.9 * len(dataset))
# #val_size = int(0.2 * len(dataset))
# val_size = len(dataset)  - train_size

# # Divide the dataset by randomly selecting samples.
# train_set, valid_set = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=1001, shuffle=False)

In [19]:
from torch.utils.data import DataLoader
train_set = CustomDataset(X_train, y_train, tokenizer)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)

In [20]:
valid_set = CustomDataset(X_train, y_train, tokenizer)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False)

In [21]:
train_set[6049]

{'text': 'забудьте указать ваше мнение содержании качество языка структуру книги изобразительные средства использованные автором другие моменты которые показались неубедительными отзыв книгу война мир льва толстого анализ слабых сторон лев толстой безусловно величайших русских писателей война мир это колоссальный труд однако несмотря признание восторги критиков книга имеет несколько слабых сторон которые взгляд заслуживают обсуждения содержание одна главных проблем войны мира это избыточность запутанность сюжета толстой стремился охватить множество персонажей включая исторические события философские размышления драматические любовные истории это приводит тому некоторые сюжетные линии теряют значимость некоторых частях книги возникает ощущение ненужной затянутости например эпизоды посвящённые наполеону порой отходят главных героев оставляя читателя недоумении соотносятся основной темой качество языка хотя стиль толстого часто восхваляется богатство глубину войне мире язык становится чре

In [22]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
epochs =10

In [24]:
from transformers import get_linear_schedule_with_warmup
scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_loader) * epochs
            )

In [25]:
loss_fn = torch.nn.CrossEntropyLoss()

In [26]:
def fit(model, train_loader, device, loss_fn, optimizer, scheduler, train_set):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in tqdm(train_loader):
        input_ids = data["input_ids"].to(device)
        attention_mask = data["attention_mask"].to(device)
        targets = data["targets"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
            )

        preds = torch.argmax(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, targets)

        correct_predictions += torch.sum(preds == targets)

        losses.append(loss.item())
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    train_acc = correct_predictions.double() / len(train_set)
    train_loss = np.mean(losses)
    return train_acc, train_loss

In [27]:
def eval(model, valid_loader, device, loss_fn, valid_set):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for data in tqdm(valid_loader):
            input_ids = data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)
            targets = data["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
    
    val_acc = correct_predictions.double() / len(valid_set)
    val_loss = np.mean(losses)
    return val_acc, val_loss

In [28]:
model.to(device)
best_accuracy = 0
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_acc, train_loss = fit(model, train_loader, device, loss_fn, optimizer, scheduler, train_set)
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval(model, valid_loader, device, loss_fn, valid_set)
    print(f'Val loss {val_loss} accuracy {val_acc}')
    print('-' * 10)

    # if val_acc > best_accuracy:
    #     torch.save(model, model_save_path)
    #     best_accuracy = val_acc

# model = torch.load(model_save_path)

Epoch 1/10


  0%|          | 0/1350 [00:00<?, ?it/s]

Train loss 0.25926221240139397 accuracy 0.9030555555555555


  0%|          | 0/1350 [00:00<?, ?it/s]

Val loss 0.19813466539968633 accuracy 0.9220833333333333
----------
Epoch 2/10


  0%|          | 0/1350 [00:00<?, ?it/s]

KeyboardInterrupt: 